<a href="https://colab.research.google.com/github/MarioRMReis/Sentinel_Images/blob/main/Saving_ROI_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to request and save an image given a region of interest.

*   First we need to acess and authenticate the EarthEngine library.

Alot of usefull information here (it's not in python):
[End-to-End Google Earth Engine](https://courses.spatialthoughts.com/end-to-end-gee.html)

In [ ]:
import ee
# Trigger the authentication flow.
ee.Authenticate()
 
# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AVHEtk5_yPDQFkoBCAEZRHp8TCncOX0om9pEjgXQcY8rl5B97hPUPXZUyiY

Successfully saved authorization token.



*   Import the required libraries

In [ ]:
import IPython.display as disp
import requests

*   If we are saving in a drive we got to mount the disired drive



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

1.  Get the region of interest. To get your region of interest you have to manually select it on the 
earth engine code editor. You can draw your shape in the map and copy the region on generated code. This generator code can be acessed in the blue button with stripes in the script. 
2.  Create the earth engine geometry
 




In [ ]:
# Region of Interest
roi = [[[-8.421062763693714, 40.20042086772547],
          [-8.421062763693714, 40.186914835987146],
          [-8.402523334982776, 40.186914835987146],
          [-8.402523334982776, 40.20042086772547]]]

roi_geometry = ee.Geometry.Polygon(roi,None,False)

1.   Create the **image collection**, for this example we are going to use 'COPERNICUS/S2' there are more you can choose from. You need to **filter the bounds** of your collection with your region's geometry
2.   Select the image you want, you can select all from a given data range. Of the image colection you can also just get the first image or a list of images you can select from. after selecting the image clip it with your geomety.
3.   To get the image we are going to use **getThumbURL** that outputs the url of the image. The image requires **min, max** and the **band** you are interested in or the **RGB**, as shown.
4.   To verify the the image requested you can use the disp command. (Increase or decrease the width to magnify the image)

In [ ]:
ImageCollection = ee.ImageCollection('COPERNICUS/S2')\
                        .filterBounds(roi_geometry)

First_Image = ee.Image(ImageCollection.first()).clip(roi_geometry)

rgb = ['B4', 'B3', 'B2']
url = First_Image.getThumbURL({"min":0, "max":2000,"bands":rgb})

disp.Image(url=url, width = 400)

*   Saving the image

Saving localy or in the drive can be done in the same fashion you just need to change the path (dont forget to mount the drive in that case). 

The id of the image is created diferently depending on the satelite and other factors, it contains usefull information as the creation date you can find more information about the one you are using online. For this one the **id** is **YYYYMMDDThhmmss_YYYYMMDDThhmmss_T29TNE** the fisrt set is the (year month day T hour minute second) start of the sensing and the last its the stop, the last set it's corresponding of the tyle or sector of the image.

Using the **requests** we can get the content of the URL, the image comes in **'bytes'**. To save it open the path with the desired name and write the data. The image type was saved in **jpg** but you can change it to others like **tiff** or **png**.



In [ ]:
path_drive = '/content/drive/MyDrive/'
path_local = 'C:/Users/mario/Documents/Mario/Sentinel'

id = First_Image.id().getInfo()

img_data = requests.get(url).content

with open(path + '/' + id +'.jpg', 'wb') as handler:
  handler.write(img_data)

*   You have other options when it comes to fitering the image collection you are creating.
1.   Filtering a start and stop date for the selection of images.
  *   As we can see the the first image of the lot is all white, it's due to the cloud coverage






In [ ]:
start_date = '2017-01-01'
end_date = '2023-12-31'

ImageCollection = ee.ImageCollection('COPERNICUS/S2')\
                        .filterBounds(roi_geometry)\
                        .filterDate(ee.Date(start_date), ee.Date(end_date))

First_Image = ee.Image(ImageCollection.first()).clip(roi_geometry)

rgb = ['B4', 'B3', 'B2']
url = First_Image.getThumbURL({"min":0, "max":2000,"bands":rgb})

disp.Image(url=url, width = 300)

*   Using the filter 'CLOUDY_PIXEL_PERCENTAGE' we can deselect the images that have a percentage of clouds higher then 10%


In [ ]:
start_date = '2017-01-01'
end_date = '2023-12-31'

ImageCollection = ee.ImageCollection('COPERNICUS/S2')\
                        .filterBounds(roi_geometry)\
                        .filterDate(ee.Date('2017-02-01'), ee.Date('2023-12-31'))\
                        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

First_Image = ee.Image(ImageCollection.first()).clip(roi_geometry)

rgb = ['B4', 'B3', 'B2']
url = First_Image.getThumbURL({"min":0, "max":2000,"bands":rgb})

disp.Image(url=url, width = 300)

*   Diferent bands have diferent optimal values of min and max.


In [ ]:
ImageCollection = ee.ImageCollection('COPERNICUS/S2')\
                        .filterDate(ee.Date('2017-08-01'), ee.Date('2023-12-31'))\
                        .filterBounds(roi_geometry)

First_Image = ee.Image(ImageCollection.first()).clip(roi_geometry)

rgb = ['B8A']
url = First_Image.getThumbURL({"min":0, "max":3000,"bands":rgb})

disp.Image(url=url, width = 300)